# Functions to move to general_utils module

In [1]:
# create a list of letters
def char_range(c1, c2):
    """Generates the characters from `c1` to `c2`, inclusive."""
    for c in range(ord(c1), ord(c2) + 1):
        yield chr(c)

# create list of wells in range
def well_range(c1,n1,c2,n2):
    well_order_by_columns = [
            f"{w}{n:1}" for n in range(n1, n2) for w in char_range(c1, c2)
        ]
    return well_order_by_columns

In [2]:
# convert ensembl to gene symbol
def convert_ensembl_symbol(adata,species='human',idcol = 'ensembls'):
    import mygene
    mg = mygene.MyGeneInfo()

    mygene_converter = mg.querymany(list(adata.var[idcol]),scopes='all', species=species, as_dataframe=True)
    mygene_converter.loc[mygene_converter['notfound']==True,'symbol'] = mygene_converter.loc[mygene_converter['notfound']==True].index

    adata.var = adata.var.merge(
        mygene_converter.reset_index(),left_on='ensembls',right_on='query').sort_values(
        by='_score',ascending=False).drop_duplicates(
        'ensembl_id').set_index('symbol')
    
    return adata

# Packages needed

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np

/home/angela/anaconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


# Clean metadata

In [3]:
tsp1_ss2_metadata_mess = pd.read_csv("../../data-objs/pilot1/TSP1_smartseq_metadata_mess.csv")
tsp1_ss2_metadata_mess.head()

,CellID,cDNAPlate,LibraryPlateID,Tissue,PrimarySort,AnatomicalPosition,SortOrder,Wells,Excluded,SortDirection,Investigator,Barcode,cDNA Prep,cDNA Dilute,library prep,lib prep date,bead cleanup,BA nM,NovaSeqRun,Additional Notes
0,NaN,B107813,B107813Blue,Lung,Epithelial,Proximal,Plate 1,A1-P12,Col24,Column,Krasnow,BC8,Bob,Roja,Bob,6/6/19,Sheela,20.2,TSPSS1Run1,NaN
1,NaN,B107813,B107813Blue,Lung,Endothelial,Proximal,Plate 1,A13-P23,Col24,Column,Krasnow,BC8,Bob,NaN,Bob,6/6/19,NaN,20.2,TSPSS1Run1,NaN
2,NaN,B107810,B107810Blue,Lung,Immune,Proximal,Plate 2,A1-P12,Col24,Column,Krasnow,BC6,Bob,NaN,Roja,6/6/19,Roja,11.8,TSPSS1Run1,NaN
3,NaN,B107810,B107810Blue,Lung,Stromal,Proximal,Plate 2,A13-P23,Col24,Column,Krasnow,BC8,Bob,NaN,Roja,6/6/19,NaN,11.8,TSPSS1Run1,NaN
4,NaN,B107809,B107809Blue,Lung,Epithelial,MedialDistal,Plate 3,A1-P12,Col24,Column,Krasnow,BC5,Bob,Roja,Roja,6/6/19,Roja,12.6,TSPSS1Run1,NaN


In [4]:
tsp1_ss2_metadata_mess.groupby(['cDNAPlate','LibraryPlateID']).count()

CellID  Tissue  PrimarySort  AnatomicalPosition  \
cDNAPlate LibraryPlateID                                                       
B107809   B107809Blue             0       2            2                   2   
B107810   B107810Blue             0       2            2                   2   
B107811   B107811Blue             0       2            2                   2   
B107813   B107813Blue             0       2            2                   2   
B107816   B107816Blue             0       1            1                   0   
B107817   B107817BlueNewTn5       0       1            1                   0   
          B107817BlueOldTn5       0       1            1                   0   
B107819   B107819Blue             0       1            1                   0   
B107820   B107820Blue             0       1            1                   0   
B107821   B107821Blue             0       1            1                   0   
B107822   B107822Blue             0       1            1                   0   
B107825   B107825Blue             0       1            1                   0   
B107826   B107826Blue             0       1            1                   0   
B107827   B107827Blue             0       1            1                   0   
B107829   B107829Blue             0       1            1                   0   
B107830   B107830Blue             0       1            1                   0   
B107909   B107909Blue             0       1            1                   0   
B107919   B107919Blue             0       1            1                   0   
B107920   B107920Blue             0       1            1                   0   
B107921   B107921Blue             0       1            1                   0   
B107924   B107924Blue             0       1            1                   0   
B107925   B107925Blue             0       1            1                   0   
B107926   B107926Blue             0       1            1                   0   

                             SortOrder  Wells  Excluded  SortDirection  \
cDNAPlate LibraryPlateID                                                 
B107809   B107809Blue                2      2         2              2   
B107810   B107810Blue                2      2         2              2   
B107811   B107811Blue                2      2         2              2   
B107813   B107813Blue                2      2         2              2   
B107816   B107816Blue                1      1         1              1   
B107817   B107817BlueNewTn5          1      1         1              1   
          B107817BlueOldTn5          1      1         1              1   
B107819   B107819Blue                1      1         1              1   
B107820   B107820Blue                1      1         1              1   
B107821   B107821Blue                1      1         1              1   
B107822   B107822Blue                1      1         1              1   
B107825   B107825Blue                1      1         1              1   
B107826   B107826Blue                1      1         1              1   
B107827   B107827Blue                1      1         1              1   
B107829   B107829Blue                1      1         1              1   
B107830   B107830Blue                1      1         1              1   
B107909   B107909Blue                1      1         1              1   
B107919   B107919Blue                1      1         1              1   
B107920   B107920Blue                1      1         1              1   
B107921   B107921Blue                1      1         1              1   
B107924   B107924Blue                1      1         1              1   
B107925   B107925Blue                1      1         1              1   
B107926   B107926Blue                1      1         1              1   

                             Investigator  Barcode  cDNA Prep  cDNA Dilute  \
cDNAPlate LibraryPlateID                                                     
B107809   B107809Blue

In [ ]:
tsp1_ss2_metadata = pd.DataFrame()

for w in tsp1_ss2_metadata_mess.index:
    waux = tsp1_ss2_metadata_mess.iloc[w]
    wauxwells = waux["Wells"]
    tsp1_ss2_metadata_plate = pd.DataFrame(well_range(
        wauxwells.split("-")[0][0],
        np.int(wauxwells.split("-")[0][1:]),
        wauxwells.split("-")[1][0],
        np.int(wauxwells.split("-")[1][1:])+1
    ), columns = ['well'])
    
    if wauxwells == 'A1-I6':
        tsp1_ss2_metadata_plate_2 = pd.DataFrame(well_range(
        "A",
        7,
        "H",
        24), columns = ['well'])
        tsp1_ss2_metadata_plate = pd.concat([tsp1_ss2_metadata_plate,tsp1_ss2_metadata_plate_2],axis=0)

    for col in tsp1_ss2_metadata_mess.columns[1:]:
        if col == "Wells":
            col_new = "Well_Range"
        else:
            col_new = col
        tsp1_ss2_metadata_plate[col_new] = waux[col]
    
    tsp1_ss2_metadata = pd.concat([tsp1_ss2_metadata,tsp1_ss2_metadata_plate],axis=0)

tsp1_ss2_metadata.to_csv("../../data-objs/pilot1/TSP1_smartseq_metadata.csv")
tsp1_ss2_metadata.head()

# Load gene count table

In [4]:
adata = sc.read_csv('/mnt/ibm_lg/angela/sapiens/TSP1/smartseq2/TSP1_gencode_realignment.csv')
adata = adata.transpose()
adata

AnnData object with n_obs × n_vars = 8037 × 58967 

Remove htseq last 5lines

In [5]:
display(adata.var_names[-10:])
adata = adata[:,:-5].copy()
display(adata.var_names[-10:])
adata

Index(['ERCC-00164', 'ERCC-00165', 'ERCC-00168', 'ERCC-00170', 'ERCC-00171',
       '__no_feature', '__ambiguous', '__too_low_aQual', '__not_aligned',
       '__alignment_not_unique'],
      dtype='object')

Index(['ERCC-00157', 'ERCC-00158', 'ERCC-00160', 'ERCC-00162', 'ERCC-00163',
       'ERCC-00164', 'ERCC-00165', 'ERCC-00168', 'ERCC-00170', 'ERCC-00171'],
      dtype='object')

AnnData object with n_obs × n_vars = 8037 × 58962 

In [6]:
adata.obs['plate'] = [c.split('_')[0] for c in adata.obs.index]
adata = adata[adata.obs['plate']!='KTRareCell'].copy()
adata

AnnData object with n_obs × n_vars = 8014 × 58962 
    obs: 'plate'

In [7]:
adata.obs['well'] = [c.split('_')[1] for c in adata.obs.index]
adata.obs.head()

,plate,well
B107809_A10_S130.homo.gencode.v30.ERCC.chrM,B107809,A10
B107809_A11_S131.homo.gencode.v30.ERCC.chrM,B107809,A11
B107809_A12_S132.homo.gencode.v30.ERCC.chrM,B107809,A12
B107809_A13_S133.homo.gencode.v30.ERCC.chrM,B107809,A13
B107809_A14_S134.homo.gencode.v30.ERCC.chrM,B107809,A14


In [8]:
adata.obs['cell_id'] = adata.obs.index
adata.obs.index = adata.obs['plate']+"_"+adata.obs['well']
adata.obs.head()

,plate,well,cell_id
B107809_A10,B107809,A10,B107809_A10_S130.homo.gencode.v30.ERCC.chrM
B107809_A11,B107809,A11,B107809_A11_S131.homo.gencode.v30.ERCC.chrM
B107809_A12,B107809,A12,B107809_A12_S132.homo.gencode.v30.ERCC.chrM
B107809_A13,B107809,A13,B107809_A13_S133.homo.gencode.v30.ERCC.chrM
B107809_A14,B107809,A14,B107809_A14_S134.homo.gencode.v30.ERCC.chrM


Convert ensembl ids to gene symbol

In [13]:
adata.var_names

Index(['ENSG00000000003.14', 'ENSG00000000005.6', 'ENSG00000000419.12',
       'ENSG00000000457.14', 'ENSG00000000460.17', 'ENSG00000000938.13',
       'ENSG00000000971.15', 'ENSG00000001036.13', 'ENSG00000001084.12',
       'ENSG00000001167.14',
       ...
       'ERCC-00157', 'ERCC-00158', 'ERCC-00160', 'ERCC-00162', 'ERCC-00163',
       'ERCC-00164', 'ERCC-00165', 'ERCC-00168', 'ERCC-00170', 'ERCC-00171'],
      dtype='object', length=58962)

In [14]:
adata.var['ensembl_id'] = adata.var_names
adata.var['ensembls'] = [g.split(".")[0] for g in adata.var['ensembl_id']]
adata.var

,ensembl_id,ensembls
ENSG00000000003.14,ENSG00000000003.14,ENSG00000000003
ENSG00000000005.6,ENSG00000000005.6,ENSG00000000005
ENSG00000000419.12,ENSG00000000419.12,ENSG00000000419
ENSG00000000457.14,ENSG00000000457.14,ENSG00000000457
ENSG00000000460.17,ENSG00000000460.17,ENSG00000000460
...,...,...
ERCC-00164,ERCC-00164,ERCC-00164
ERCC-00165,ERCC-00165,ERCC-00165
ERCC-00168,ERCC-00168,ERCC-00168
ERCC-00170,ERCC-00170,ERCC-00170


In [19]:
adata = convert_ensembl_symbol(adata);

# Combine raw data with metadata

In [10]:
metadata = pd.read_csv("../../data-objs/pilot1/TSP1_smartseq_metadata.csv",index_col=0)
metadata.head()

,well,cDNAPlate,LibraryPlateID,Tissue,PrimarySort,AnatomicalPosition,SortOrder,Well_Range,Excluded,SortDirection,Investigator,Barcode,cDNA Prep,cDNA Dilute,library prep,lib prep date,bead cleanup,BA nM,NovaSeqRun,Additional Notes
0,A1,B107813,B107813Blue,Lung,Epithelial,Proximal,Plate 1,A1-P12,Col24,Column,Krasnow,BC8,Bob,Roja,Bob,6/6/19,Sheela,20.2,TSPSS1Run1,NaN
1,B1,B107813,B107813Blue,Lung,Epithelial,Proximal,Plate 1,A1-P12,Col24,Column,Krasnow,BC8,Bob,Roja,Bob,6/6/19,Sheela,20.2,TSPSS1Run1,NaN
2,C1,B107813,B107813Blue,Lung,Epithelial,Proximal,Plate 1,A1-P12,Col24,Column,Krasnow,BC8,Bob,Roja,Bob,6/6/19,Sheela,20.2,TSPSS1Run1,NaN
3,D1,B107813,B107813Blue,Lung,Epithelial,Proximal,Plate 1,A1-P12,Col24,Column,Krasnow,BC8,Bob,Roja,Bob,6/6/19,Sheela,20.2,TSPSS1Run1,NaN
4,E1,B107813,B107813Blue,Lung,Epithelial,Proximal,Plate 1,A1-P12,Col24,Column,Krasnow,BC8,Bob,Roja,Bob,6/6/19,Sheela,20.2,TSPSS1Run1,NaN


In [11]:
metadata.index = metadata['cDNAPlate']+"_"+metadata['well']

In [12]:
adata.obs = adata.obs.join(
    metadata,
    lsuffix='_seq', 
    rsuffix='_metadata',
    how='left'
).drop_duplicates(
    ['plate','well_seq','cDNAPlate']
)
adata.obs.head()

In [17]:
adata

AnnData object with n_obs × n_vars = 8014 × 58962 
    obs: 'plate', 'well_seq', 'cell_id', 'well_metadata', 'cDNAPlate', 'LibraryPlateID', 'Tissue', 'PrimarySort', 'AnatomicalPosition', 'SortOrder', 'Well_Range', 'Excluded', 'SortDirection', 'Investigator', 'Barcode', 'cDNA Prep', 'cDNA Dilute', 'library prep', 'lib prep date', 'bead cleanup', 'BA nM', 'NovaSeqRun', 'Additional Notes'
    var: 'ensembl_id', 'ensembls', 'query', '_id', '_score', 'entrezgene', 'name', 'taxid', 'notfound'

Write the new object

In [16]:
adata.write_h5ad('/mnt/ibm_lg/angela/sapiens/TSP1/smartseq2/TSP1_SS2_gencode.h5ad')
adata

... storing 'plate' as categorical
... storing 'well_seq' as categorical
... storing 'well_metadata' as categorical
... storing 'cDNAPlate' as categorical
... storing 'LibraryPlateID' as categorical
... storing 'Tissue' as categorical
... storing 'PrimarySort' as categorical
... storing 'AnatomicalPosition' as categorical
... storing 'SortOrder' as categorical
... storing 'Well_Range' as categorical
... storing 'Excluded' as categorical
... storing 'SortDirection' as categorical
... storing 'Investigator' as categorical
... storing 'Barcode' as categorical
... storing 'cDNA Prep' as categorical
... storing 'cDNA Dilute' as categorical
... storing 'library prep' as categorical
... storing 'lib prep date' as categorical
... storing 'bead cleanup' as categorical
... storing 'NovaSeqRun' as categorical
... storing 'Additional Notes' as categorical
... storing 'ensembls' as categorical
... storing 'query' as categorical
... storing '_id' as categorical
... storing 'entrezgene' as categorica

AnnData object with n_obs × n_vars = 8014 × 58962 
    obs: 'plate', 'well_seq', 'cell_id', 'well_metadata', 'cDNAPlate', 'LibraryPlateID', 'Tissue', 'PrimarySort', 'AnatomicalPosition', 'SortOrder', 'Well_Range', 'Excluded', 'SortDirection', 'Investigator', 'Barcode', 'cDNA Prep', 'cDNA Dilute', 'library prep', 'lib prep date', 'bead cleanup', 'BA nM', 'NovaSeqRun', 'Additional Notes'
    var: 'ensembl_id', 'ensembls', 'query', '_id', '_score', 'entrezgene', 'name', 'taxid', 'notfound'